# Import Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

In [3]:
# Reading the data
data = pd.read_excel('../data/responses_data.xlsx')

In [4]:
# Checking the head of the data
data.head()

,question_id,description,num_response,label_in_question,answer_id,answer_category_num,answer_justification,answer_upvote,account_id_id,question_id_id,date
0,4,ما فيك تعيش بلا ما تكب فتن ليل نهار وبكرة قلهم...,0,normal,147,Normal,none were given,0,52,4,2022-12-03
1,47,هؤلاء الخبثاء والخنازير أينما حلو يحل بارضهم ا...,0,hate,76,Violent,none were given,0,8,47,2022-11-19
2,2,سورية بلد الحضارات تربطها بعلية او بحيوان,2,normal,11,Racist,none were given,0,1,2,2022-09-09
3,2,سورية بلد الحضارات تربطها بعلية او بحيوان,2,normal,17,Normal,none were given,0,7,2,2022-09-19
4,2,سورية بلد الحضارات تربطها بعلية او بحيوان,2,normal,16,Religious affiliation,none were given,0,6,2,2022-09-19


In [6]:
data.shape

(195, 11)

In [7]:
data.description = data.description.apply(str.strip)

In [8]:
# The shape of the data
data.shape

(195, 11)

In [9]:
# The columns of the data
data.columns

Index(['question_id', 'description', 'num_response', 'label_in_question',
       'answer_id', 'answer_category_num', 'answer_justification',
       'answer_upvote', 'account_id_id', 'question_id_id', 'date'],
      dtype='object')

In [10]:
# checking how many comments are repeated
i = 0
for comment in data.description.unique():
    temp_df = data[data.description == comment]
    if temp_df.shape[0] >= 2:
        
        i+=1
print(i)

5


In [11]:
clean_data = data.copy()
clean_data  = clean_data[['description', 'answer_category_num']]
clean_data.head(2)

,description,answer_category_num
0,ما فيك تعيش بلا ما تكب فتن ليل نهار وبكرة قلهم...,Normal
1,هؤلاء الخبثاء والخنازير أينما حلو يحل بارضهم ا...,Violent


In [13]:
clean_data.description.value_counts()

وزير الخارجية اللبناني جبران باسيل قال في سلسلة تغريداته عقب اختتام القمة العربية التنموية الاقتصادية والاجتماعية لمسنا تج…    7
سورية بلد الحضارات تربطها بعلية او بحيوان                                                                                      6
هيدا ايراني فارسي يهودي من احفاد القرده و الخنازير                                                                             2
اخي الحاج اذا شعرت انك محرجا من الانتقادات لتصريح جبران باسيل لا داعي لأن تهاجم المنتقدين                                      2
جعجع يقصف عمرك ياحشاش ياعميل ياوسخ خراس يبليك باتلك روح انضب يا زبالة                                                          2
                                                                                                                              ..
سوف يتحاسبو جميعالان الحق سلطان                                                                                                1
طيب يا بغلة                                                                                      

In [9]:
# removing duplicated descrptions/ comments
clean_data.drop_duplicates(subset='description', inplace=True)
clean_data.reset_index(drop=True, inplace=True)

In [10]:
clean_data

,description,answer_category_num
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
1,سورية بلد الحضارات تربطها بعلية او بحيوان,Violent
2,تقتلون وسام الحسن وتترحموعلية من أي أصناف المخ...,Racist
3,معك خبر انو بلدة قطر متل ما سميتا مساحتها اكبر...,Normal
4,للامانه قوت الموسم اللي طاف كان هوا بس متحمس ح...,Normal
...,...,...
70,ولك خلص بقى، خلص والله ارحم حالك، هيدي الصفات ...,Violent
71,زمن الاوغاد,Mockery
72,ما أوطاك,Mockery
73,خليت صرمايته لكم ياتسلاب ايران,Violent


In [11]:
# checking how many comments have different votes
for i, row in clean_data.iterrows():

    comment = row.description

    # getting the dataframe for that comment
    temp_df = data[data.description == comment]
    
    # how many unique answers that comment has
    n_ = temp_df.answer_category_num.nunique()

    # changing the answer_category_num to the most frequent one
    if n_ >= 2:
        most_voted_label = temp_df.answer_category_num.value_counts().index[0]
        clean_data.at[i, 'answer_category_num'] = most_voted_label

In [12]:
clean_data

,description,answer_category_num
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
1,سورية بلد الحضارات تربطها بعلية او بحيوان,Violent
2,تقتلون وسام الحسن وتترحموعلية من أي أصناف المخ...,Racist
3,معك خبر انو بلدة قطر متل ما سميتا مساحتها اكبر...,Normal
4,للامانه قوت الموسم اللي طاف كان هوا بس متحمس ح...,Normal
...,...,...
70,ولك خلص بقى، خلص والله ارحم حالك، هيدي الصفات ...,Violent
71,زمن الاوغاد,Mockery
72,ما أوطاك,Mockery
73,خليت صرمايته لكم ياتسلاب ايران,Violent


In [13]:
# Generated Class
clean_data.answer_category_num.value_counts()

Normal                   21
Mockery                  21
Violent                  19
Religious affiliation     7
Racist                    7
Name: answer_category_num, dtype: int64

In [14]:
# Checking the info
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   description          75 non-null     object
 1   answer_category_num  75 non-null     object
dtypes: object(2)
memory usage: 1.3+ KB


We can see that there is no missing values

In [15]:
# Removing stop words for each tweet
clean_data['tweet_no_stopwords'] = 'x'
stop_words = set(stopwords.words('arabic')) 
for count, tweet in enumerate(clean_data.description):
    word_tokens = word_tokenize(tweet)
    filtered_tweet = []
    for word in word_tokens:
        if word not in stop_words:
            filtered_tweet.append(word)
    joined_filtered_tweet = " ".join(filtered_tweet)
    clean_data.tweet_no_stopwords[count] = joined_filtered_tweet

In [16]:
clean_data

,description,answer_category_num,tweet_no_stopwords
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation,وزير الخارجية اللبناني جبران باسيل قال سلسلة ت...
1,سورية بلد الحضارات تربطها بعلية او بحيوان,Violent,سورية بلد الحضارات تربطها بعلية او بحيوان
2,تقتلون وسام الحسن وتترحموعلية من أي أصناف المخ...,Racist,تقتلون وسام الحسن وتترحموعلية أصناف المخلوقات
3,معك خبر انو بلدة قطر متل ما سميتا مساحتها اكبر...,Normal,معك خبر انو بلدة قطر متل سميتا مساحتها اكبر لب...
4,للامانه قوت الموسم اللي طاف كان هوا بس متحمس ح...,Normal,للامانه قوت الموسم اللي طاف هوا متحمس حق الجاي...
...,...,...,...
70,ولك خلص بقى، خلص والله ارحم حالك، هيدي الصفات ...,Violent,ولك خلص بقى، خلص والله ارحم حالك، هيدي الصفات ...
71,زمن الاوغاد,Mockery,زمن الاوغاد
72,ما أوطاك,Mockery,أوطاك
73,خليت صرمايته لكم ياتسلاب ايران,Violent,خليت صرمايته ياتسلاب ايران


# Training the Model

In [17]:
# y is Class which is dependent on X Tweet
X = clean_data['tweet_no_stopwords']
y = clean_data['answer_category_num']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [18]:
y_train.value_counts()

Normal                   21
Violent                  18
Mockery                  16
Religious affiliation     6
Racist                    6
Name: answer_category_num, dtype: int64

In [19]:
y_test.value_counts()

Mockery                  5
Religious affiliation    1
Racist                   1
Violent                  1
Name: answer_category_num, dtype: int64

In [20]:
# Using tf-idf 
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train)
X_train_tfidf.shape

(67, 492)

In [21]:
# Using pipiles for machine learning flow

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),])


text_clf.fit(X_train, y_train)  


predictions = text_clf.predict(X_test)

In [22]:
# Confusion Matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[2 2 0 0 1]
 [0 0 0 0 0]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 1 0 0 0]]


In [23]:
# Classification report
print(metrics.classification_report(y_test,predictions)) 

                       precision    recall  f1-score   support

              Mockery       1.00      0.40      0.57         5
               Normal       0.00      0.00      0.00         0
               Racist       0.00      0.00      0.00         1
Religious affiliation       0.00      0.00      0.00         1
              Violent       0.00      0.00      0.00         1

             accuracy                           0.25         8
            macro avg       0.20      0.08      0.11         8
         weighted avg       0.62      0.25      0.36         8



c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

In [24]:
# Accuracy
print(metrics.accuracy_score(y_test,predictions))

0.25


In [25]:
tst =  "انت  واحد حقير وما بتستحي ا"

In [26]:
text_clf.predict([tst])[0]

'Mockery'

In [27]:
for count, test_tweet in enumerate(X_test):
    print(count)
    print("Tweet: {}".format(test_tweet))
    print("Real Prediction: {} ||  Model Prediction: {}".format(y_test.values[count], predictions[count]))

0
Tweet: العمى شو تشتوش
Real Prediction: Mockery ||  Model Prediction: Violent
1
Tweet: كبير ماعم نشوفك عم تجيب سيرة السعودية والامارات
Real Prediction: Mockery ||  Model Prediction: Normal
2
Tweet: انتي مسخرة بشرفي
Real Prediction: Mockery ||  Model Prediction: Mockery
3
Tweet: وئام وهاب كلب يحركة رائحة الدسم المال لاعقيدة ولاعروبه وابعد مايكون الرجوله والشرف
Real Prediction: Mockery ||  Model Prediction: Mockery
4
Tweet: وزير الخارجية اللبناني جبران باسيل قال سلسلة تغريداته عقب اختتام القمة العربية التنموية الاقتصادية والاجتماعية لمسنا تج…
Real Prediction: Religious affiliation ||  Model Prediction: Normal
5
Tweet: كلام سليم صلاة الجمعه يدعون حكام الخليج والخليج دافع مليارات عشات قضيتهم
Real Prediction: Racist ||  Model Prediction: Violent
6
Tweet: اعتقد ان مارسيل غانم حيوان
Real Prediction: Mockery ||  Model Prediction: Normal
7
Tweet: الخبثاء والخنازير حلو يحل بارضهم الخراب
Real Prediction: Violent ||  Model Prediction: Normal


# Saving the Model

In [27]:
import pickle
# save the model to disk
filename = 'finalized_model_SVC.sav'
pickle.dump(text_clf, open(filename, 'wb'))

In [74]:
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.8393162393162393
